In [10]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile
from qiskit.circuit.library.standard_gates import PhaseGate, MCPhaseGate


import numpy as np
pi = np.pi
backend = Aer.get_backend('qasm_simulator')

In [11]:
def IQPE_circ(qc0, U, ϕs, k):
    """The quantum circuit for the IQPE
    
    Args:
        qc0: Quantum circuit for the initial state |ψ>
        U: The gate used to estimate the phase
        ϕs: The digits previously measured
        k: The k-th iteration
        
    Note: 
        We don't include the circuit that create the state |ψ>
    
    """
    n = qc0.num_qubits
    qr = QuantumRegister(n+1)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)
#     qc = QuantumCircuit(qr)
    
    qc.append(qc0, qr[1:])
    
    ωk = 0
    for i, ϕ0 in enumerate(ϕs):
        ωk += ϕ0 * 2**(-(i+2))
    
    ωk = -2*pi * ωk
      
    qc.h(0)  
    p = 2**(k-1)

    Up = U.power(p).control(1, label=f"U^{p}") 
    
    qc.append(Up, qr)
    
    qc.rz(ωk, 0)
    qc.h(0)
    qc.measure(0, 0)
    
    return qc

def IQPE(qc0, U, m, backend = Aer.get_backend('qasm_simulator'), shots = 1000):
    """Use IQPE to estimate ϕ.
    
    Args:
        qc0: Quantum circuit for the initial state |ψ>
        U: The gate used to estimate the phase
        m: The number of digits for ϕ 
        backend: The backend used for IQPE
        shots: number of shots
    """
    
    n = qc0.num_qubits
    qr = QuantumRegister(n+1)
    qc = QuantumCircuit(qr)
    qc.append(qc0, qr[1:])

    ϕs = []
    counts_list = []
    for k in range(m, 0, -1):
#         qc.append(IQPE_circ(n, U, ϕs, k), qr)
        qc = IQPE_circ(qc0, U, ϕs, k)
        
        qc_transpile = transpile(qc, backend=backend, seed_transpiler=42, optimization_level=3)
        counts = backend.run(qc_transpile, shots = shots).result().get_counts()
        counts_list.append(counts)
        if '1' in counts:
            ϕs.insert(0, 1)
        else:
            ϕs.insert(0, 0)
        
    return ϕs, counts_list, qc
    

In [12]:
m = 10
ϕs = [0] * int(m/2) + [1] * (m-int(m/2))
np.random.shuffle(ϕs)
ϕ = sum([ϕ0 * 2**(-i-1) for i, ϕ0 in enumerate(ϕs)])

print(f"ϕ, ϕs={ϕ, ϕs}")

n = 4

if n==1:
    U = PhaseGate(2*pi*ϕ)
else:
    U = MCPhaseGate(2*pi*ϕ, n-1)

qc0 = QuantumCircuit(n, name = 'Xs')
for i in range(qc0.num_qubits):
    qc0.x(i)
    
ϕs, counts_list, qc = IQPE(qc0, U, m)
print(f"n, m, ϕs={n, m, ϕs}")
print(f"counts_list={counts_list}")
qc.draw()

ϕ, ϕs=(0.720703125, [1, 0, 1, 1, 1, 0, 0, 0, 1, 0])
n, m, ϕs=(4, 10, [1, 0, 1, 1, 1, 0, 0, 0, 1, 0])
counts_list=[{'0': 1000}, {'1': 1000}, {'0': 1000}, {'0': 1000}, {'0': 1000}, {'1': 1000}, {'1': 1000}, {'1': 1000}, {'0': 1000}, {'1': 1000}]


┌───┐      U^1      ┌─────────────┐┌───┐┌─┐
q2357_0: ─┤ H ├───────■───────┤ Rz(-1.3867) ├┤ H ├┤M├
         ┌┴───┴┐┌─────┴──────┐└─────────────┘└───┘└╥┘
q2357_1: ┤0    ├┤0           ├─────────────────────╫─
         │     ││            │                     ║ 
q2357_2: ┤1    ├┤1           ├─────────────────────╫─
         │  Xs ││  mcphase^1 │                     ║ 
q2357_3: ┤2    ├┤2           ├─────────────────────╫─
         │     ││            │                     ║ 
q2357_4: ┤3    ├┤3           ├─────────────────────╫─
         └─────┘└────────────┘                     ║ 
  c19: 1/══════════════════════════════════════════╩═
                                                   0

In [13]:
a = range(9)

In [14]:
list(a)

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [15]:
for k in range(len(a), 0, -1):
    print(k, list(a[k:]))

9 []
8 [8]
7 [7, 8]
6 [6, 7, 8]
5 [5, 6, 7, 8]
4 [4, 5, 6, 7, 8]
3 [3, 4, 5, 6, 7, 8]
2 [2, 3, 4, 5, 6, 7, 8]
1 [1, 2, 3, 4, 5, 6, 7, 8]


In [17]:
qrU = QuantumRegister(n)
qcU = QuantumCircuit(n)
qcU

AttributeError: 'QuantumCircuit' object has no attribute 'U'